<a href="https://colab.research.google.com/github/rsidorchuk93/audio/blob/main/Lex_Fridman_podcast_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lex Fridman podcast transcripts analysis

Transcripts data downloaded from https://karpathy.ai/lexicap/

In [1]:
# connecting to Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# importing libraries
import pandas as pd
import os
import re

In [4]:
folder = "/content/drive/My Drive/text/lex_fridman_vtt/"

In [115]:
os.listdir(folder)[:10]

['episode_232_small.vtt',
 'episode_232_large.vtt',
 'episode_231_small.vtt',
 'episode_231_large.vtt',
 'episode_230_large.vtt',
 'episode_229_small.vtt',
 'episode_228_small.vtt',
 'episode_228_large.vtt',
 'episode_230_small.vtt',
 'episode_229_large.vtt']

In [114]:
file_path = '/content/drive/My Drive/text/lex_fridman_vtt/episode_189_large.vtt'
# Open the VTT file and read its contents
with open('/content/drive/My Drive/text/lex_fridman_vtt/episode_189_large.vtt', 'r') as file:
    contents = file.read()

# Print the resulting text content
contents[:1000]

"WEBVTT\n\n00:00.000 --> 00:02.760\n The following is a conversation with David Sinclair.\n\n00:02.760 --> 00:06.200\n He's a professor in the Department of Genetics at Harvard\n\n00:06.200 --> 00:08.920\n and co director of the Paul F. Glenn Center\n\n00:08.920 --> 00:12.280\n for the Biology of Aging at Harvard Medical School.\n\n00:12.280 --> 00:13.280\n He's the author of the book,\n\n00:13.280 --> 00:16.840\n Lifespan and co founder of several biotech companies.\n\n00:16.840 --> 00:20.800\n He works on turning age into an engineering problem\n\n00:20.800 --> 00:22.160\n and solving it.\n\n00:22.160 --> 00:23.960\n Driven by a vision of a world\n\n00:23.960 --> 00:25.960\n where billions of people can live much longer\n\n00:25.960 --> 00:28.160\n and much healthier lives.\n\n00:28.160 --> 00:30.360\n Quick mention of our sponsors,\n\n00:30.360 --> 00:33.400\n Onnit, Clear, National Instruments,\n\n00:33.400 --> 00:36.840\n and I, SimpliSafe and Linode.\n\n00:36.840 --> 00:40.040\n 

In [47]:
def remove_timestamps_vtt(file_path):
    with open(file_path, 'r') as file:
      content = file.read()

    # Remove "WEBVTT" header
    content = content.replace("WEBVTT", "").strip()
    
    # Remove timestamps
    cleaned_content = re.sub(r'(\d{1,2}:)?\d{1,2}:\d{2}\.\d{3} --> (\d{1,2}:)?\d{1,2}:\d{2}\.\d{3}', '', content)

    # Remove extra newline characters
    cleaned_content = re.sub(r'\n{2,}', '\n', cleaned_content)

    # Replace newline characters with spaces
    cleaned_content = cleaned_content.replace('\n', ' ')

    # Remove extra whitespaces
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content)

    return cleaned_content

In [113]:
cleaned_content = remove_timestamps_vtt(file_path)
cleaned_content[:1000]

" The following is a conversation with David Sinclair. He's a professor in the Department of Genetics at Harvard and co director of the Paul F. Glenn Center for the Biology of Aging at Harvard Medical School. He's the author of the book, Lifespan and co founder of several biotech companies. He works on turning age into an engineering problem and solving it. Driven by a vision of a world where billions of people can live much longer and much healthier lives. Quick mention of our sponsors, Onnit, Clear, National Instruments, and I, SimpliSafe and Linode. Check them out in the description to support this podcast. As a side note, let me say that longevity research challenges us to think how science and engineering will change society. Imagine if we can live 100,000 years, even under controlled conditions, like in a spaceship say, then suddenly a trip to Alpha Centauri that is a 4.37 light years away takes a single human lifespan. And on the psychological, maybe even philosophical level, as

In [116]:
# number of characters
len(cleaned_content)

94352

In [51]:
# split by word
len(cleaned_content.split())

17456

In [53]:
# split by sentence
len(re.split(r'[.!?]+', cleaned_content))

1188

In [99]:
# split into chunks so that we could summarize each chunk
def split_text(text, max_length):
    tokens = tokenizer.encode(text)
    num_chunks = (len(tokens) + max_length - 1) // max_length
    chunks = []

    for i in range(num_chunks):
        start = i * max_length
        end = (i + 1) * max_length
        chunk_tokens = tokens[start:end]
        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)
    
    return chunks

max_length = 1024
chunks = split_text(cleaned_content, max_length)

In [101]:
len(chunks)

21

In [55]:
pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.1 MB/s eta 0:00:00


In [92]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [105]:
def summarize(text, tokenizer, model):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [106]:
# Summarize the text
summary = summarize(chunks[0], tokenizer, model)
print(summary)

David Sinclair is a professor in the Department of Genetics at Harvard Medical School. He's the author of the book, Lifespan and co founder of several biotech companies. Sinclair: I usually feel like the same person when I was 12. I've still got that boy in me and I can look at things as I would a six-year-old. I have to, on most days, just pretend to be a grownup and be mature, but I definitely don't feel that way.


In [107]:
summarized_chunks = []
for chunk in chunks:
    inputs = tokenizer(chunk, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, min_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summarized_chunks.append(summary)

summarized_text = ' '.join(summarized_chunks)

In [110]:
summarized_chunks

['David Sinclair is a professor in the Department of Genetics at Harvard Medical School. He\'s the author of the book, Lifespan and co founder of several biotech companies. He works on turning age into an engineering problem and solving it. Sinclair: "I usually feel like the same person when I was 12. Like when I, right now, as I think about myself, I feel like exactly the sameperson that I was when I were 12" "I\'ve still got that boy in me and I can look at things," he says.',
 'The writer-director has been working on a project about bringing people back to life through artificial intelligence. He says he\'d like to bring back people that mean something to us personally, not just Einstein, but people that we\'ve lost and in that way achieve a kind of small artificial immortality. "I wouldn\'t go back in time. I would, but I\'d prefer to bring him into the future and say, can you believe we have this thinking machine in our pockets now?" he says.',
 "David Perry: I'm a biologist. I fi

In [111]:
len(summarized_text)

10387